<a href="https://colab.research.google.com/github/saraspreitzhofer/knapsack/blob/main/WFP2_Sara_Spreitzhofer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initialization

In [44]:
pip install pyeasyga

Note: you may need to restart the kernel to use updated packages.


In [45]:
import random
import time
from pyeasyga import pyeasyga

In [46]:
# constants
NBR_ITEMS = 250       # number of randomly generated items
MAX_PROFIT = 100      # maximum profit of each item
MAX_WEIGHT = 50       # maximum weight of each item
MAX_VOLUME = 50       # maximum volume of each item
MAX_SPACE = 50        # maximum capacity of each item
DIMENSION = 10        # dimensions of the knapsack
NBR_EXPERIMENTS = 1   # number of experiments for statistical evaluation
NBR_KNAPSACKS = 5     # number of knapsacks for the multiple multidimensional knapsack problem

# constants for the genetic algorithm
NGEN = 75             # number of generations
MU = 150              # population size
CXPB = 0.7            # crossover probability
MUTPB = 0.2           # mutation probability

In [47]:
# To assure reproducibility, the RNG seed is set prior to the items dict initialization
random.seed(64)

In [48]:
# function to create random 1-dimensional items
def create_items():
  # Create a dictionary of NBR_ITEMS random items to map the values and weights.
  # Create the item dictionary: item name is an integer, and value is a (value, weight) 2-tuple.
  items = {}
  # Create random items and store them in the items' dictionary.
  for i in range(NBR_ITEMS):
      items[i] = (random.randint(1, MAX_PROFIT), random.randint(1, 15))  # or random.uniform(0, 100)
      # print("For item no. ",i+1,", Value is: ",items[i][0]," and Weight is: ",items[i][1])
  return items

In [49]:
# function to create random 2-dimensional items
def create_items_2d():
  # Create a dictionary of NBR_ITEMS random items to map the values, weights and volumes.
  # Create the item dictionary: item name is an integer, and value is a (value, weight, volume) 3-tuple.
  items2d = {}
  # Create random items and store them in the items' dictionary.
  for i in range(NBR_ITEMS):
      items2d[i] = (random.randint(1, MAX_PROFIT), random.randint(1, 15), random.randint(1, 10))
      # print("For item no. ",i+1,", Value is: ",items2d[i][0], ", Weight is: ",items2d[i][1], " and Volume is: ",items2d[i][2])
  # print(items2d)
  return items2d

In [50]:
# function to create random n-dimensional items
def create_items_nd():
  # Create a dictionary of NBR_ITEMS random items to map the values, ....
  # Create the item dictionary: item name is an integer, and value is a (value, ...) n-tuple.
  itemsnd = {}
  # Create random items and store them in the items' dictionary.
  for i in range(NBR_ITEMS):
    values = []
    values.append(random.randint(1, MAX_PROFIT))  # value (= dimension 0)
    for j in range(1, DIMENSION+1):
      values.append(random.randint(1, 15)) # dimensions 1 to n
    itemsnd[i] = values
  # print(itemsnd)
  return itemsnd

In [51]:
# function to calculate the runtime of any function specified as parameter
def calcTime(function, *args):
  start = time.perf_counter()         # get current time    
  function(*args)
  end = time.perf_counter()
  elapsed = (end - start) * 100       # compute time value in milliseconds
  elapsed = round(elapsed, 5)         
  # print("Elapsed time until solved: " + str(elapsed) + " milliseconds")
  return elapsed

# Greedy Algorithm, 1-dimensional 0-1 Knapsack
The implementation of the greedy algorithm is based on https://www.kaggle.com/code/lordv15/greedy-knapsack-problem.

In [52]:
def heuristic01(x):
  return x[0]           # profit

def heuristic02(x):
  return x[1]           # weight

def heuristic03(x):
  return x[0]/x[1]      # profit/weight

In [53]:
def GreedyKnapsack(NBR_ITEMS, items, heuristic, mybool):
    # sorted list in descending order of the specified heuristic function
    items_sorted = sorted(items.values(), key=lambda x: heuristic(x), reverse=mybool)

    available_weight = MAX_WEIGHT
    knapsack = []
    maxprofit = 0

    for i in range(NBR_ITEMS):
      if items_sorted[i][1] <= available_weight:
        knapsack.insert(i, items_sorted[i])
        maxprofit = maxprofit + items_sorted[i][0]
        available_weight = available_weight - items_sorted[i][1]
    # print("Knapsack: ", knapsack)
    # print("Max Profit: ", maxprofit)
    # print("Used weight: ", MAX_WEIGHT - available_weight, " from ", MAX_WEIGHT)
    return maxprofit

In [54]:
# statistical evaluation to compare runtime and maximum profit of the different heuristics

time1 = 0
time2 = 0
time3 = 0
profit1 = 0
profit2 = 0
profit3 = 0

for i in range (NBR_EXPERIMENTS):
  items = create_items()
  time1 = time1 + calcTime(GreedyKnapsack, NBR_ITEMS, items, heuristic01, True)
  time2 = time2 + calcTime(GreedyKnapsack, NBR_ITEMS, items, heuristic02, False)
  time3 = time3 + calcTime(GreedyKnapsack, NBR_ITEMS, items, heuristic03, True)
  profit1 = profit1 + GreedyKnapsack(NBR_ITEMS, items, heuristic01, True)
  profit2 = profit2 + GreedyKnapsack(NBR_ITEMS, items, heuristic02, False)
  profit3 = profit3 + GreedyKnapsack(NBR_ITEMS, items, heuristic03, True)

time1 = time1 / NBR_EXPERIMENTS
time2 = time2 / NBR_EXPERIMENTS
time3 = time3 / NBR_EXPERIMENTS
profit1 = profit1 / NBR_EXPERIMENTS
profit2 = profit2 / NBR_EXPERIMENTS
profit3 = profit3 / NBR_EXPERIMENTS

print("Statistical evaluation of 1-dimensional greedy 0-1 knapsack")
print("\t\tmillisec\tmaxprofit\t")
print("GKS01\t", time1, "\t", profit1, " \t")
print("GKS02\t", time2, "\t", profit2, " \t")
print("GKS03\t", time3, "\t", profit3, " \t")

Statistical evaluation of 1-dimensional greedy 0-1 knapsack
		millisec	maxprofit	
GKS01	 0.0157 	 786.0  	
GKS02	 0.00865 	 1848.0  	
GKS03	 0.01091 	 2157.0  	


# Greedy Algorithm, 2-dimensional 0-1 Knapsack

In [55]:
def heuristic1(x):
  return x[0]                     # profit

def heuristic2(x):
  return x[0]/(x[1] + x[2])       # profit / (weight + volume)

def heuristic3(x):
  return x[0]/x[1] + x[0]/x[2]    # profit / weight + profit / volume

In [56]:
def GreedyKnapsack_2d(heuristic, NBR_ITEMS, items2d):
    # sorted list in descending order of the specified heuristic function
    items_sorted = sorted(items2d.values(), key=lambda x: (heuristic(x)), reverse=True)

    available_weight = MAX_WEIGHT
    available_volume = MAX_VOLUME
    knapsack = []
    maxprofit = 0
    for i in range(NBR_ITEMS):
      if items_sorted[i][1] <= available_weight and items_sorted[i][2] <= available_volume:
        knapsack.append(items_sorted[i])
        maxprofit = maxprofit + items_sorted[i][0]
        available_weight = available_weight - items_sorted[i][1]
        available_volume = available_volume - items_sorted[i][2]
    usedweight = MAX_WEIGHT - available_weight
    usedvolume = MAX_VOLUME - available_volume
    '''print("GreedyKnapsack: ", knapsack)
    print("Max Profit: ", maxprofit)
    print("Used weight: ", usedweight, " from ", MAX_WEIGHT)
    print("Used volume: ", usedvolume, " from ", MAX_VOLUME)'''
    return maxprofit, usedweight, usedvolume

In [57]:
# statistical evaluation to compare runtime and maximum profit of the different heuristics

avgtimeGKS1 = 0
avgtimeGKS2 = 0
avgtimeGKS3 = 0
resultsGKS1 = [0,0,0] # maxprofit, usedweight, usedvolume
resultsGKS2 = [0,0,0]
resultsGKS3 = [0,0,0]

for j in range (NBR_EXPERIMENTS):
  items2d = create_items_2d()

  # get time
  elapsed1 = calcTime(GreedyKnapsack_2d,heuristic1,NBR_ITEMS,items2d)
  elapsed2 = calcTime(GreedyKnapsack_2d,heuristic2,NBR_ITEMS,items2d)
  elapsed3 = calcTime(GreedyKnapsack_2d,heuristic3,NBR_ITEMS,items2d)
  avgtimeGKS1 = avgtimeGKS1 + elapsed1
  avgtimeGKS2 = avgtimeGKS2 + elapsed2
  avgtimeGKS3 = avgtimeGKS3 + elapsed3

  result1 = GreedyKnapsack_2d(heuristic1,NBR_ITEMS,items2d)
  result2 = GreedyKnapsack_2d(heuristic2,NBR_ITEMS,items2d)
  result3 = GreedyKnapsack_2d(heuristic3,NBR_ITEMS,items2d)
  # get maxprofit
  resultsGKS1[0] = resultsGKS1[0] + result1[0]
  resultsGKS2[0] = resultsGKS2[0] + result2[0]
  resultsGKS3[0] = resultsGKS3[0] + result3[0]
  # get usedweight
  resultsGKS1[1] = resultsGKS1[1] + result1[1]
  resultsGKS2[1] = resultsGKS2[1] + result2[1]
  resultsGKS3[1] = resultsGKS3[1] + result3[1]
  # get usedvolume
  resultsGKS1[2] = resultsGKS1[2] + result1[2]
  resultsGKS2[2] = resultsGKS2[2] + result2[2]
  resultsGKS3[2] = resultsGKS3[2] + result3[2]

avgtimeGKS1 = round(avgtimeGKS1 / NBR_EXPERIMENTS, 5)
avgtimeGKS2 = round(avgtimeGKS2 / NBR_EXPERIMENTS, 5)
avgtimeGKS3 = round(avgtimeGKS3 / NBR_EXPERIMENTS, 5)

avgprofit1 = round(resultsGKS1[0] / NBR_EXPERIMENTS, 6)
avgprofit2 = round(resultsGKS2[0] / NBR_EXPERIMENTS, 5)
avgprofit3 = round(resultsGKS3[0] / NBR_EXPERIMENTS, 5)
avgweight1 = round(resultsGKS1[1] / NBR_EXPERIMENTS, 5)
avgweight2 = round(resultsGKS2[1] / NBR_EXPERIMENTS, 5)
avgweight3 = round(resultsGKS3[1] / NBR_EXPERIMENTS, 5)
avgvolume1 = round(resultsGKS1[2] / NBR_EXPERIMENTS, 5)
avgvolume2 = round(resultsGKS2[2] / NBR_EXPERIMENTS, 5)
avgvolume3 = round(resultsGKS3[2] / NBR_EXPERIMENTS, 5)

print("Statistical evaluation of 2-dimensional greedy 0-1 knapsack")
print("\t\tmillisec\tmaxprofit\tusedweight\tusedvolume")
print("GKS1\t", avgtimeGKS1, "\t", avgprofit1, " \t", avgweight1, "  \t", avgvolume1, "\t")
print("GKS2\t", avgtimeGKS2, "\t", avgprofit2, " \t", avgweight2, "  \t", avgvolume2, "\t")
print("GKS3\t", avgtimeGKS3, "\t", avgprofit3, " \t", avgweight3, "  \t", avgvolume3, "\t")

Statistical evaluation of 2-dimensional greedy 0-1 knapsack
		millisec	maxprofit	usedweight	usedvolume
GKS1	 0.00881 	 696.0  	 50.0   	 39.0 	
GKS2	 0.01105 	 1436.0  	 50.0   	 46.0 	
GKS3	 0.01162 	 1263.0  	 50.0   	 48.0 	


# Greedy Algorithm, n-dimensional 0-1 Knapsack


In [58]:
def heuristic4(x):
  return x[0], x[1], x[2], x[3], x[4]   # value, then dimension1, 2, 3, 4

def heuristic5(x):  # x is itemsnd[0], itemsnd[1], ...
    sum = 0
    for i in range(1, DIMENSION):
      sum = sum + x[i]
    return x[0]/sum                     # value / sum(dimensions)

def heuristic6(x):  # x is itemsnd[0], itemsnd[1], ...
    sum = 0
    for i in range(1, DIMENSION):
      sum = sum + x[i]/MAX_SPACE
    return x[0]/sum                     # value / sum(dimensions/capacity)

In [59]:
def GreedyKnapsack_nd(heuristic, nbr_items, dimensions, itemsnd):
    # sorted list in descending order of heuristic function
    items_sorted = sorted(itemsnd.values(), key=lambda x: (heuristic(x)), reverse=True)
    
    available_space = []    # free space for each dimension
    for i in range(dimensions):
      available_space.append(MAX_SPACE)
    knapsack = []
    maxprofit = 0
    error_count = 0
    
    for i in range(nbr_items):
      for j in range(dimensions): # in the n-dimensional knapsack problem, each item has 1 value & n properties
        if items_sorted[i][j + 1] > available_space[j]:
          error_count = 1
          break
      if error_count == 0:
        knapsack.append(items_sorted[i])
        maxprofit = maxprofit + items_sorted[i][0]
        for j in range(dimensions):
          available_space[j] = available_space[j] - items_sorted[i][j+1]
    
    usedspace = []
    for i in range(dimensions):
      usedspace.append(MAX_SPACE - available_space[i])
    # print("GreedyKnapsack: ", knapsack)
    # print("Max Profit: ", maxprofit)
    avg_used_space = 0
    for i in range(dimensions):
      avg_used_space = avg_used_space + usedspace[i]
      # print("Used space in dimension ", i+1, ": ", usedspace[i], " from ", MAX_SPACE)
    avg_used_space = round(avg_used_space / dimensions, 5)
    # print("average used space:", avg_used_space)
    return maxprofit, avg_used_space, usedspace

In [60]:
# statistical evaluation to compare runtime and maximum profit of the different heuristics

avgtimeGKS4 = 0
avgtimeGKS5 = 0
avgtimeGKS6 = 0
resultsGKS4 = [0,0] # maxprofit, avg_used_space
resultsGKS5 = [0,0]
resultsGKS6 = [0,0]

for j in range (NBR_EXPERIMENTS):
  itemsnd = create_items_nd()

  # get time
  elapsed4 = calcTime(GreedyKnapsack_nd, heuristic4, NBR_ITEMS, DIMENSION, itemsnd)
  elapsed5 = calcTime(GreedyKnapsack_nd, heuristic5, NBR_ITEMS, DIMENSION, itemsnd)
  elapsed6 = calcTime(GreedyKnapsack_nd, heuristic6, NBR_ITEMS, DIMENSION, itemsnd)
  avgtimeGKS4 = avgtimeGKS4 + elapsed4
  avgtimeGKS5 = avgtimeGKS5 + elapsed5
  avgtimeGKS6 = avgtimeGKS6 + elapsed6

  result4 = GreedyKnapsack_nd(heuristic4,NBR_ITEMS,DIMENSION,itemsnd)
  result5 = GreedyKnapsack_nd(heuristic5,NBR_ITEMS,DIMENSION,itemsnd)
  result6 = GreedyKnapsack_nd(heuristic6,NBR_ITEMS,DIMENSION,itemsnd)
  # get maxprofit
  resultsGKS4[0] = resultsGKS4[0] + result4[0]
  resultsGKS5[0] = resultsGKS5[0] + result5[0]
  resultsGKS6[0] = resultsGKS6[0] + result6[0]
  # get average used space
  resultsGKS4[1] = resultsGKS4[1] + result4[1]
  resultsGKS5[1] = resultsGKS5[1] + result5[1]
  resultsGKS6[1] = resultsGKS6[1] + result6[1]

avgtimeGKS4 = round(avgtimeGKS4 / NBR_EXPERIMENTS, 5)
avgtimeGKS5 = round(avgtimeGKS5 / NBR_EXPERIMENTS, 5)
avgtimeGKS6 = round(avgtimeGKS6 / NBR_EXPERIMENTS, 5)

avgprofit4 = round(resultsGKS4[0] / NBR_EXPERIMENTS, 5)
avgprofit5 = round(resultsGKS5[0] / NBR_EXPERIMENTS, 5)
avgprofit6 = round(resultsGKS6[0] / NBR_EXPERIMENTS, 5)
avgspace4 = round(resultsGKS4[1] / NBR_EXPERIMENTS, 5)
avgspace5 = round(resultsGKS5[1] / NBR_EXPERIMENTS, 5)
avgspace6 = round(resultsGKS6[1] / NBR_EXPERIMENTS, 5)

print("Statistical evaluation of n-dimensional greedy 0-1 knapsack")
print("\t\tmillisec\tmaxprofit\taverage used space")
print("GKS4\t", avgtimeGKS4, "\t", avgprofit4, "\t\t", avgspace4, "\t")
print("GKS5\t", avgtimeGKS5, "\t", avgprofit5, "\t\t", avgspace5, "\t")
print("GKS6\t", avgtimeGKS6, "\t", avgprofit6, "\t\t", avgspace6, "\t")

Statistical evaluation of n-dimensional greedy 0-1 knapsack
		millisec	maxprofit	average used space
GKS4	 0.03891 	 393.0 		 33.4 	
GKS5	 0.04357 	 541.0 		 35.7 	
GKS6	 0.05184 	 541.0 		 35.7 	


# Greedy Algorithm, multiple n-dimensional 0-1 Knapsack

In [61]:
# sorted list in descending order of value/sum(dimensions)
def heuristic(x):  # x is itemsnd[0], itemsnd[1], ...
    sum = 0
    for i in range(1, DIMENSION):
      sum = sum + x[i]
    return x[0]/sum

In [62]:
def GreedyKnapsack_multiple(heuristic, nbr_items, dimensions, itemsnd, nbr_knapsacks):
    # sorted list in descending order of heuristic function
    items_sorted = sorted(itemsnd.values(), key=lambda x: (heuristic(x)), reverse=True)

    available_space = []    # free space for each dimension for each knapsack
    for k in range(nbr_knapsacks):
      knapack_space = []
      for i in range(dimensions):
        knapack_space.append(MAX_SPACE)
      available_space.append(knapack_space)

    multiple_knapsacks = []
    total_maxprofit = 0

    for k in range(nbr_knapsacks):
      knapsack = []
      maxprofit = 0
      unused_items = []   # unused items are items for other knapsacks, so each item is only selected once

      for i in range(nbr_items):
        error_count = 0
        for j in range(dimensions): # in the n-dimensional knapsack problem, each item has 1 value & n properties
          if items_sorted[i][j + 1] > available_space[k][j]:
            error_count = 1
            unused_items.append(items_sorted[i])
            break
        if error_count == 0:
          knapsack.append(items_sorted[i])
          maxprofit = maxprofit + items_sorted[i][0]
          for j in range(dimensions):
            available_space[k][j] = available_space[k][j] - items_sorted[i][j+1]
      multiple_knapsacks.append(knapsack)
      total_maxprofit = total_maxprofit + maxprofit
      items_sorted = unused_items
      nbr_items = len(unused_items)
      # print("knapsack", knapsack)
      # print("unused_items", unused_items)
    
    # print("multiple_knapsacks", multiple_knapsacks)
    # print("available space", available_space)
    # print("total maxprofit", total_maxprofit)

    total_usedspace = []
    for k in range(nbr_knapsacks):
      usedspace = []
      for i in range(dimensions):
        usedspace.append(MAX_SPACE - available_space[k][i])
      total_usedspace.append(usedspace)
    # print("GreedyKnapsack: ", knapsack)
    # print("Max Profit: ", maxprofit)
    avg_used_space = 0
    for k in range(nbr_knapsacks):
      for i in range(dimensions):
        avg_used_space = avg_used_space + total_usedspace[k][i]
      avg_used_space = avg_used_space / dimensions
    avg_used_space = round(avg_used_space / nbr_knapsacks, 5)
    # print("average used space:", avg_used_space)
    return total_maxprofit, avg_used_space, total_usedspace

In [63]:
avgtimeGKS = 0
resultsGKS = [0,0] # maxprofit, avg_used_space

for j in range (NBR_EXPERIMENTS):
  print(j+1)
  itemsnd = create_items_nd()

  # get time
  elapsed = calcTime(GreedyKnapsack_multiple, heuristic, NBR_ITEMS, DIMENSION, itemsnd, NBR_KNAPSACKS)
  avgtimeGKS = avgtimeGKS + elapsed
  result = GreedyKnapsack_multiple(heuristic,NBR_ITEMS,DIMENSION,itemsnd, NBR_KNAPSACKS)
  # get maxprofit
  resultsGKS[0] = resultsGKS[0] + result[0]
  # get average used space
  resultsGKS[1] = resultsGKS[1] + result[1]

avgtimeGKS = round(avgtimeGKS / NBR_EXPERIMENTS, 5)
avgprofit = round(resultsGKS[0] / NBR_EXPERIMENTS, 5)
avgspace = round(resultsGKS[1] / NBR_EXPERIMENTS, 5)

print("Statistical evaluation of multiple n-dimensional greedy 0-1 knapsack")
print("\t\t\t\tmillisec\tmaxprofit\taverage used space")
print("multiple GKS\t", avgtimeGKS, "\t", avgprofit, "\t\t", avgspace, "\t")

1
Statistical evaluation of multiple n-dimensional greedy 0-1 knapsack
				millisec	maxprofit	average used space
multiple GKS	 0.11127 	 2493.0 		 6.98449 	


# Genetic Algorithm, 1-dimensional 0-1 Knapsack with PYEASYGA
The implementation of the one-dimensional genetic algorithm is inspired by https://pyeasyga.readthedocs.io/en/latest/examples.html#dimensional-knapsack-problem. 

In [64]:
# define a fitness function
def fitness_1d(individual, data):  # individual = candidate solution (array of 0s & 1s)
  values, weights = 0, 0
  for (selected, item) in zip(individual, data):  # selected = 0 or 1
    if selected and (weights + item[1]) <= MAX_WEIGHT:
      values += item[0]
      weights += item[1]
  if weights > MAX_WEIGHT:
    values = 0
  return values, weights    # maxprofit, used space 

In [65]:
def geneticAlgEASYsimple(data):
  ga = pyeasyga.GeneticAlgorithm(data)  # initialise the GA with data
    # population_size=50,
    # generations=100,
    # crossover_probability=0.8,
    # mutation_probability=0.2

  ga.fitness_function = fitness_1d               # set the GA's fitness function
  ga.run()                                    # run the GA
  # print("best individual", ga.best_individual())  # print the GA's best solution
  return ga.best_individual()                 # returns (best.fitness, best.genes)

In [66]:
def geneticAlgEASY(data):
  ga = pyeasyga.GeneticAlgorithm(         # initialise the GA with data & more
          data,
          population_size=MU,             # number of individuals to select for the next generation
          generations=NGEN,               # number of generations      
          crossover_probability=CXPB,     # probability that an offspring is produced by crossover
          mutation_probability=MUTPB,     # probability that an offspring is produced by mutation
          elitism=True,
          maximise_fitness=True)

  ga.fitness_function = fitness_1d               # set the GA's fitness function
  ga.run()                                    # run the GA
  # print("best individual", ga.best_individual())  # print the GA's best solution
  return ga.best_individual()                 # returns (best.fitness, best.genes)

In [67]:
# statistical evaluation to compare two different parameter combinations for the genetic algorithm

avgtimePYEASYGAsimple = 0
avgtimePYEASYGA = 0
resultsPYEASYGAsimple = [0,0]       # best.fitness, best.genes
resultsPYEASYGA = [0,0]

for j in range (NBR_EXPERIMENTS):
  my_data = create_items().values()

  # get time
  elapsed1 = calcTime(geneticAlgEASYsimple, my_data)
  elapsed2 = calcTime(geneticAlgEASY, my_data)
  avgtimePYEASYGAsimple = avgtimePYEASYGAsimple + elapsed1
  avgtimePYEASYGA = avgtimePYEASYGA + elapsed2

  result1 = geneticAlgEASYsimple(my_data)
  result2 = geneticAlgEASY(my_data)
  # get maxprofit
  resultsPYEASYGAsimple[0] = resultsPYEASYGAsimple[0] + result1[0][0]
  resultsPYEASYGA[0] = resultsPYEASYGA[0] + result2[0][0]
  # get usedweight
  resultsPYEASYGAsimple[1] = resultsPYEASYGAsimple[1] + result1[0][1]
  resultsPYEASYGA[1] = resultsPYEASYGA[1] + result2[0][1]
  
avgtimePYEASYGAsimple = round(avgtimePYEASYGAsimple / NBR_EXPERIMENTS, 5)
avgtimePYEASYGA = round(avgtimePYEASYGA / NBR_EXPERIMENTS, 5)

avgprofitPYEASYGAsimple = round(resultsPYEASYGAsimple[0] / NBR_EXPERIMENTS, 5)
avgprofitPYEASYGA = round(resultsPYEASYGA[0] / NBR_EXPERIMENTS, 5)
avgweightPYEASYGAsimple = round(resultsPYEASYGAsimple[1] / NBR_EXPERIMENTS, 5)
avgweightPYEASYGA = round(resultsPYEASYGA[1] / NBR_EXPERIMENTS, 5)

print("Statistical evaluation of 1-dimensional genetic 0-1 knapsack (PYEASYGA)")
print("\t\t\t\tmillisec\tmaxprofit\tusedweight")
print("PYEASYGA simple\t", avgtimePYEASYGAsimple, "\t", avgprofitPYEASYGAsimple, " \t", avgweightPYEASYGAsimple, "  \t")
print("PYEASYGA\t\t", avgtimePYEASYGA, "\t", avgprofitPYEASYGA, " \t", avgweightPYEASYGA, "  \t")

Statistical evaluation of 1-dimensional genetic 0-1 knapsack (PYEASYGA)
				millisec	maxprofit	usedweight
PYEASYGA simple	 90.66264 	 888.0  	 50.0   	
PYEASYGA		 214.49042 	 747.0  	 50.0   	


# Genetic Algorithm, n-dimensional 0-1 Knapsack with PYEASYGA
The implementation of the n-dimensional genetic algorithm is inspired by https://pyeasyga.readthedocs.io/en/latest/examples.html#multi-dimensional-knapsack-problem.

In [68]:
# define a fitness function
def fitness(individual, data):    # individual = array of 0s & 1s
    used_space = []
    profit_array = []
    error = 0
    for i in range(DIMENSION+1):
      used_space.append(0)
    for (selected, item) in zip(individual, data):  # selected = 0 or 1
        if selected:
            for i in range(1, DIMENSION+1):
              if (used_space[i]+item[i]) > MAX_SPACE:   # if one dimension is higher that the capacity -> error = 1
                error = 1                  
              if error < 1:
                used_space[i] += item[i]
            if error < 1:
              used_space[0] += item[0]                # profit
              profit_array.append(item[0])
            else:
               profit_array.append(0)
               error = 0

    avg_used_space = 0
    for i in range (1, DIMENSION+1):
      avg_used_space += used_space[i]
    avg_used_space = round(avg_used_space / DIMENSION, 5)

    return used_space[0], avg_used_space, used_space[1:], profit_array  # maxprofit, avg_used_space, used_space, profit for each item

In [69]:
# crossover probability = 0.8, pop size = 150, generations = 75, mutation probability = 0.2
def genetigAlgEASYnd1(data):
  ga = pyeasyga.GeneticAlgorithm(data)        # initialise the GA with data
  ga.population_size = 150                    # increase population size to 150 (default value is 50)
  ga.generations=NGEN                         # decrease generations to 75 (default value is 100)
  ga.fitness_function = fitness               # set the GA's fitness function
  ga.run()                                    # run the GA
  # print("best individual 1", ga.best_individual())  # print the GA's best solution
  # print("1: maxprofit", ga.best_individual()[0][0], ", profit", ga.best_individual()[0][3])
  return ga.best_individual()                # returns (best.fitness (maxprofit, avg_used_space, used_space, profit), best.genes)

In [70]:
# crossover probability = 0.7, pop size = 150, generations = 75, mutation probability = 0.2
def genetigAlgEASYnd2(data):
  ga = pyeasyga.GeneticAlgorithm(         # initialise the GA with data & more
          data,
          population_size=MU,             # number of individuals to select for the next generation
          generations=NGEN,               # number of generations
          crossover_probability=CXPB,     # probability that an offspring is produced by crossover
          mutation_probability=MUTPB,     # probability that an offspring is produced by mutation
          elitism=True,
          maximise_fitness=True)
  ga.fitness_function = fitness               # set the GA's fitness function
  ga.run()                                    # run the GA
  # print("best individual 2", ga.best_individual())  # print the GA's best solution
  # print("2: maxprofit", ga.best_individual()[0][0], ", profit", ga.best_individual()[0][3])
  return ga.best_individual()                # returns (best.fitness (maxprofit, avg_used_space, used_space, profit), best.genes)

In [71]:
# statistical evaluation to compare two different parameter combinations for the genetic algorithm

avgtimePYEASYnd1 = 0
avgtimePYEASYnd2 = 0
resultsPYEASYnd1 = [0,0]       # profit, avg_used_space
resultsPYEASYnd2 = [0,0]

for j in range (NBR_EXPERIMENTS):
  my_data_nd = create_items_nd().values()

  # get time
  elapsed1 = calcTime(genetigAlgEASYnd1, my_data_nd)
  elapsed2 = calcTime(genetigAlgEASYnd2, my_data_nd)
  avgtimePYEASYnd1 = avgtimePYEASYnd1 + elapsed1
  avgtimePYEASYnd2 = avgtimePYEASYnd2 + elapsed2
  
  result1 = genetigAlgEASYnd1(my_data_nd)
  result2 = genetigAlgEASYnd2(my_data_nd)
  # get maxprofit
  resultsPYEASYnd1[0] = resultsPYEASYnd1[0] + result1[0][0]
  resultsPYEASYnd2[0] = resultsPYEASYnd2[0] + result2[0][0]
  # get average used space
  resultsPYEASYnd1[1] = resultsPYEASYnd1[1] + result1[0][1]
  resultsPYEASYnd2[1] = resultsPYEASYnd2[1] + result2[0][1]
  
avgtimePYEASYnd1 = round(avgtimePYEASYnd1 / NBR_EXPERIMENTS, 5)
avgtimePYEASYnd2 = round(avgtimePYEASYnd2 / NBR_EXPERIMENTS, 5)
avgprofitPYEASYnd1 = round(resultsPYEASYnd1[0] / NBR_EXPERIMENTS, 5)
avgprofitPYEASYnd2 = round(resultsPYEASYnd2[0] / NBR_EXPERIMENTS, 5)
usedspacePYEASYnd1 = round(resultsPYEASYnd1[1] / NBR_EXPERIMENTS, 5)
usedspacePYEASYnd2 = round(resultsPYEASYnd2[1] / NBR_EXPERIMENTS, 5)

print("Statistical evaluation of n-dimensional genetic 0-1 knapsack (PYEASYGA)")
print("\t\t\t\tmillisec\tmaxprofit\taverage used space")
print("PYEASYGA nd 1\t", avgtimePYEASYnd1, "\t", avgprofitPYEASYnd1, " \t", usedspacePYEASYnd1)
print("PYEASYGA nd 2\t", avgtimePYEASYnd2, "\t", avgprofitPYEASYnd2, " \t", usedspacePYEASYnd2)

Statistical evaluation of n-dimensional genetic 0-1 knapsack (PYEASYGA)
				millisec	maxprofit	average used space
PYEASYGA nd 1	 1007.0053 	 434.0  	 39.7
PYEASYGA nd 2	 776.12735 	 434.0  	 39.7


# Improved pyeasyga module
The existing code of the pyeasyga module is modified to improve the genetic algorithm.

In [72]:
# -*- coding: utf-8 -*-
"""
    improved pyeasyga module: class MyGeneticAlgorithm

"""

import random
import copy
from operator import attrgetter

from six.moves import range


class MyGeneticAlgorithm(object):

    def __init__(self,
                 seed_data,
                 max_capacity,    # for the improved create_individual function
                 number_of_dimensions=1,    # for the improved create_individual function, n-dimensional
                 population_size=50,
                 generations=100,
                 crossover_probability=0.8,
                 mutation_probability=0.2,
                 elitism=True,
                 maximise_fitness=True):
        """Instantiate the Genetic Algorithm.

        :param seed_data: input data to the Genetic Algorithm
        :type seed_data: list of objects
        :param int population_size: size of population
        :param int generations: number of generations to evolve
        :param float crossover_probability: probability of crossover operation
        :param float mutation_probability: probability of mutation operation

        """

        self.seed_data = seed_data
        self.max_capacity = max_capacity    # for the improved create_individual function
        self.number_of_dimensions = number_of_dimensions    # for the improved create_individual function, n-dimensional
        self.population_size = population_size
        self.generations = generations
        self.crossover_probability = crossover_probability
        self.mutation_probability = mutation_probability
        self.elitism = elitism
        self.maximise_fitness = maximise_fitness

        self.current_generation = []

        def create_individual(seed_data):
            """Create a candidate solution representation.

            e.g. for a bit array representation:

            >>> return [random.randint(0, 1) for _ in range(len(data))]

            :param seed_data: input data to the Genetic Algorithm
            :type seed_data: list of objects
            :returns: candidate solution representation as a list

            """
            # return [random.randint(0, 1) for _ in range(len(seed_data))]

            feasible = True
            individual_list = []
            random.shuffle(seed_data)
            
            # 1-dimensional
            '''capacity = 0

            for item in seed_data:
              r = random.randint(0, 1)
              if(r):
                capacity = capacity + item[1]
              if capacity > self.max_capacity:
                individual_list.append(0)
                break
              else:
                individual_list.append(r)

            for i in range(len(seed_data)-len(individual_list)):
              individual_list.append(0)'''

            # n-dimensional
            capacity = []
            for i in range(number_of_dimensions):
              capacity.append(0)

            for item in seed_data:
              r = random.randint(0, 1)
              if r:
                for i in range(number_of_dimensions):
                  capacity[i] = capacity[i] + item[i+1]
                  if capacity[i] > self.max_capacity:
                    individual_list.append(0)
                    feasible = False
                    break
              if not feasible:
                break
              individual_list.append(r)

            for i in range(len(seed_data)-len(individual_list)):
              individual_list.append(0)

            return individual_list

        def crossover(parent_1, parent_2):
            """Crossover (mate) two parents to produce two children.

            :param parent_1: candidate solution representation (list)
            :param parent_2: candidate solution representation (list)
            :returns: tuple containing two children

            """
            index = random.randrange(1, len(parent_1))
            child_1 = parent_1[:index] + parent_2[index:]
            child_2 = parent_2[:index] + parent_1[index:]
            return child_1, child_2

        def mutate(individual):
            """Reverse the bit of a random index in an individual."""
            mutate_index = random.randrange(len(individual))
            individual[mutate_index] = (0, 1)[individual[mutate_index] == 0]

        def random_selection(population):
            """Select and return a random member of the population."""
            return random.choice(population)

        def tournament_selection(population):
            """Select a random number of individuals from the population and
            return the fittest member of them all.
            """
            if self.tournament_size == 0:
                self.tournament_size = 2
            members = random.sample(population, self.tournament_size)
            members.sort(
                key=attrgetter('fitness'), reverse=self.maximise_fitness)
            return members[0]

        self.fitness_function = None
        self.tournament_selection = tournament_selection
        self.tournament_size = self.population_size // 10
        self.random_selection = random_selection
        self.create_individual = create_individual
        self.crossover_function = crossover
        self.mutate_function = mutate
        self.selection_function = self.tournament_selection

    def create_initial_population(self):
        """Create members of the first population randomly.
        """
        initial_population = []
        for _ in range(self.population_size):
            genes = self.create_individual(self.seed_data)
            individual = Chromosome(genes)
            initial_population.append(individual)
        self.current_generation = initial_population

    def calculate_population_fitness(self):
        """Calculate the fitness of every member of the given population using
        the supplied fitness_function.
        """
        for individual in self.current_generation:
            individual.fitness = self.fitness_function(
                individual.genes, self.seed_data)

    def rank_population(self):
        """Sort the population by fitness according to the order defined by
        maximise_fitness.
        """
        self.current_generation.sort(
            key=attrgetter('fitness'), reverse=self.maximise_fitness)

    def create_new_population(self):
        """Create a new population using the genetic operators (selection,
        crossover, and mutation) supplied.
        """
        new_population = []
        elite = copy.deepcopy(self.current_generation[0])
        selection = self.selection_function

        while len(new_population) < self.population_size:
            parent_1 = copy.deepcopy(selection(self.current_generation))
            parent_2 = copy.deepcopy(selection(self.current_generation))

            child_1, child_2 = parent_1, parent_2
            child_1.fitness, child_2.fitness = 0, 0

            can_crossover = random.random() < self.crossover_probability
            can_mutate = random.random() < self.mutation_probability

            if can_crossover:
                child_1.genes, child_2.genes = self.crossover_function(
                    parent_1.genes, parent_2.genes)

            if can_mutate:
                self.mutate_function(child_1.genes)
                self.mutate_function(child_2.genes)

            new_population.append(child_1)
            if len(new_population) < self.population_size:
                new_population.append(child_2)

        if self.elitism:
            new_population[0] = elite

        self.current_generation = new_population

    def create_first_generation(self):
        """Create the first population, calculate the population's fitness and
        rank the population by fitness according to the order specified.
        """
        self.create_initial_population()
        self.calculate_population_fitness()
        self.rank_population()

    def create_next_generation(self):
        """Create subsequent populations, calculate the population fitness and
        rank the population by fitness in the order specified.
        """
        self.create_new_population()
        self.calculate_population_fitness()
        self.rank_population()

    def run(self):
        """Run (solve) the Genetic Algorithm."""
        self.create_first_generation()

        for _ in range(1, self.generations):
            self.create_next_generation()

    def best_individual(self):
        """Return the individual with the best fitness in the current
        generation.
        """
        best = self.current_generation[0]
        return (best.fitness, best.genes)

    def last_generation(self):
        """Return members of the last generation as a generator function."""
        return ((member.fitness, member.genes) for member
                in self.current_generation)


class Chromosome(object):
    """ Chromosome class that encapsulates an individual's fitness and solution
    representation.
    """
    def __init__(self, genes):
        """Initialise the Chromosome."""
        self.genes = genes
        self.fitness = 0

    def __repr__(self):
        """Return initialised Chromosome representation in human readable form.
        """
        return repr((self.fitness, self.genes))

# Improved GA, 1-dimensional 0-1 Knapsack

In [73]:
def geneticAlgEASYimproved(data):
  ga = MyGeneticAlgorithm(         # initialise the GA with data & more
          data,
          max_capacity=MAX_SPACE,       # for the improved create_individual function
          population_size=MU,             # number of individuals to select for the next generation
          generations=NGEN,               # number of generations      
          crossover_probability=0.8,     # probability that an offspring is produced by crossover
          mutation_probability=MUTPB,     # probability that an offspring is produced by mutation
          elitism=True,
          maximise_fitness=True)

  ga.fitness_function = fitness_1d               # set the GA's fitness function
  ga.run()                                    # run the GA
  # print("best individual", ga.best_individual())  # print the GA's best solution
  return ga.best_individual()                 # returns (best.fitness, best.genes)

In [74]:
# statistical evaluation to compare the genetic algorithm with the improved genetic algorithm

avgtimePYEASYGA = 0
avgtimePYEASYGAimproved = 0
resultsPYEASYGA = [0,0]       # best.fitness, best.genes
resultsPYEASYGAimproved = [0,0]

for j in range (NBR_EXPERIMENTS):
  print(j+1)
  my_data = create_items().values()

  # get time
  elapsed1 = calcTime(geneticAlgEASY, my_data)
  elapsed2 = calcTime(geneticAlgEASYimproved, list(my_data))
  avgtimePYEASYGA = avgtimePYEASYGA + elapsed1
  avgtimePYEASYGAimproved = avgtimePYEASYGAimproved + elapsed2

  result1 = geneticAlgEASY(my_data)
  result2 = geneticAlgEASYimproved(list(my_data))
  # get maxprofit
  resultsPYEASYGA[0] = resultsPYEASYGA[0] + result1[0][0]
  resultsPYEASYGAimproved[0] = resultsPYEASYGAimproved[0] + result2[0][0]
  # get usedweight
  resultsPYEASYGA[1] = resultsPYEASYGA[1] + result1[0][1]
  resultsPYEASYGAimproved[1] = resultsPYEASYGAimproved[1] + result2[0][1]
  
avgtimePYEASYGA = round(avgtimePYEASYGA / NBR_EXPERIMENTS, 5)
avgtimePYEASYGAimproved = round(avgtimePYEASYGAimproved / NBR_EXPERIMENTS, 5)

avgprofitPYEASYGA = round(resultsPYEASYGA[0] / NBR_EXPERIMENTS, 5)
avgprofitPYEASYGAimproved = round(resultsPYEASYGAimproved[0] / NBR_EXPERIMENTS, 5)
avgweightPYEASYGA = round(resultsPYEASYGA[1] / NBR_EXPERIMENTS, 5)
avgweightPYEASYGAimproved = round(resultsPYEASYGAimproved[1] / NBR_EXPERIMENTS, 5)

print("Statistical evaluation of 1-dimensional genetic 0-1 knapsack (PYEASYGA & improved GA)")
print("\t\t\tmillisec\tmaxprofit\tusedweight")
print("PYEASYGA\t", avgtimePYEASYGA, "\t", avgprofitPYEASYGA, " \t", avgweightPYEASYGA, "  \t")
print("improved GA\t", avgtimePYEASYGAimproved, "\t", avgprofitPYEASYGAimproved, " \t", avgweightPYEASYGAimproved, "  \t")

1
Statistical evaluation of 1-dimensional genetic 0-1 knapsack (PYEASYGA & improved GA)
			millisec	maxprofit	usedweight
PYEASYGA	 216.8036 	 669.0  	 50.0   	
improved GA	 205.4091 	 751.0  	 50.0   	


# Improved GA, n-dimensional 0-1 Knapsack

In [75]:
def geneticAlgEASYimproved_nd(data):
  ga = MyGeneticAlgorithm(         # initialise the GA with data & more
          data,
          max_capacity=MAX_SPACE,       # for the improved create_individual function
          population_size=MU,             # number of individuals to select for the next generation
          generations=NGEN,               # number of generations      
          crossover_probability=0.8,     # probability that an offspring is produced by crossover
          mutation_probability=MUTPB,     # probability that an offspring is produced by mutation
          elitism=True,
          maximise_fitness=True)

  ga.fitness_function = fitness               # set the GA's fitness function
  ga.run()                                    # run the GA
  # print("best individual", ga.best_individual())  # print the GA's best solution
  return ga.best_individual()                 # returns (best.fitness, best.genes)

In [76]:
# statistical evaluation to compare the genetic algorithm with the improved genetic algorithm

avgtimePYEASYGA_nd = 0
avgtimePYEASYGAimproved_nd = 0
resultsPYEASYGA_nd = [0,0]       # best.fitness, best.genes
resultsPYEASYGAimproved_nd = [0,0]

for j in range (NBR_EXPERIMENTS):
  print(j+1)
  my_data = create_items_nd().values()

  # get time
  elapsed1 = calcTime(genetigAlgEASYnd1, my_data)
  elapsed2 = calcTime(geneticAlgEASYimproved_nd, list(my_data))
  avgtimePYEASYGA_nd = avgtimePYEASYGA_nd + elapsed1
  avgtimePYEASYGAimproved_nd = avgtimePYEASYGAimproved_nd + elapsed2

  result1 = genetigAlgEASYnd1(my_data)
  result2 = geneticAlgEASYimproved_nd(list(my_data))
  # get maxprofit
  resultsPYEASYGA_nd[0] = resultsPYEASYGA_nd[0] + result1[0][0]
  resultsPYEASYGAimproved_nd[0] = resultsPYEASYGAimproved_nd[0] + result2[0][0]
  # get usedweight
  resultsPYEASYGA_nd[1] = resultsPYEASYGA_nd[1] + result1[0][1]
  resultsPYEASYGAimproved_nd[1] = resultsPYEASYGAimproved_nd[1] + result2[0][1]
  
avgtimePYEASYGA_nd = round(avgtimePYEASYGA_nd / NBR_EXPERIMENTS, 5)
avgtimePYEASYGAimproved_nd = round(avgtimePYEASYGAimproved_nd / NBR_EXPERIMENTS, 5)

avgprofitPYEASYGA_nd = round(resultsPYEASYGA_nd[0] / NBR_EXPERIMENTS, 5)
avgprofitPYEASYGAimproved_nd = round(resultsPYEASYGAimproved_nd[0] / NBR_EXPERIMENTS, 5)
avgweightPYEASYGA_nd = round(resultsPYEASYGA_nd[1] / NBR_EXPERIMENTS, 5)
avgweightPYEASYGAimproved_nd = round(resultsPYEASYGAimproved_nd[1] / NBR_EXPERIMENTS, 5)

print("Statistical evaluation of n-dimensional genetic 0-1 knapsack (PYEASYGA & improved GA)")
print("\t\tmillisec\tmaxprofit\tusedweight")
print("PYEASYGA\t", avgtimePYEASYGA_nd, "\t", avgprofitPYEASYGA_nd, " \t", avgweightPYEASYGA_nd, "  \t")
print("improved GA\t", avgtimePYEASYGAimproved_nd, "\t", avgprofitPYEASYGAimproved_nd, " \t", avgweightPYEASYGAimproved_nd, "  \t")

1
Statistical evaluation of n-dimensional genetic 0-1 knapsack (PYEASYGA & improved GA)
		millisec	maxprofit	usedweight
PYEASYGA	 646.00655 	 424.0  	 43.2   	
improved GA	 354.40292 	 300.0  	 44.3   	


# Pyeasyga module for multiple knapsack
The existing code of the pyeasyga module is modified for the multiple multidimensional knapsack problem.

In [77]:
# -*- coding: utf-8 -*-
"""
    pyeasyga module for multiple knapsack: class MultipleGeneticAlgorithm

"""

import random
import copy
from operator import attrgetter

from six.moves import range


class MultipleGeneticAlgorithm(object):
    """Genetic Algorithm class.

    This is the main class that controls the functionality of the Genetic
    Algorithm.

    A simple example of usage:

    >>> # Select only two items from the list and maximise profit
    >>> from pyeasyga.pyeasyga import GeneticAlgorithm
    >>> input_data = [('pear', 50), ('apple', 35), ('banana', 40)]
    >>> easyga = GeneticAlgorithm(input_data)
    >>> def fitness (member, data):
    >>>     return sum([profit for (selected, (fruit, profit)) in
    >>>                 zip(member, data) if selected and
    >>>                 member.count(1) == 2])
    >>> easyga.fitness_function = fitness
    >>> easyga.run()
    >>> print easyga.best_individual()

    """

    def __init__(self,
                 seed_data,
                 knapsacks=1,  # for multiple multidimensional knapsack
                 population_size=50,
                 generations=100,
                 crossover_probability=0.8,
                 mutation_probability=0.2,
                 elitism=True,
                 maximise_fitness=True):
        """Instantiate the Genetic Algorithm.

        :param seed_data: input data to the Genetic Algorithm
        :type seed_data: list of objects
        :param int population_size: size of population
        :param int generations: number of generations to evolve
        :param float crossover_probability: probability of crossover operation
        :param float mutation_probability: probability of mutation operation

        """

        self.seed_data = seed_data
        self.knapsacks = knapsacks  # for multiple multidimensional knapsack
        self.population_size = population_size
        self.generations = generations
        self.crossover_probability = crossover_probability
        self.mutation_probability = mutation_probability
        self.elitism = elitism
        self.maximise_fitness = maximise_fitness

        self.current_generation = []

        def create_individual(seed_data):
            """Create a candidate solution representation.

            e.g. for a bit array representation:

            >>> return [random.randint(0, 1) for _ in range(len(data))]

            :param seed_data: input data to the Genetic Algorithm
            :type seed_data: list of objects
            :returns: candidate solution representation as a list

            """
            # print("individual_list with duplicates")
            individual_list = []
            for i in range(knapsacks):
                knapsack_list = []
                for j in range(len(seed_data)):
                    knapsack_list.append(random.randint(0, 1))
                # print(knapsack_list)
                individual_list.append(knapsack_list)

            # print("individual_list without duplicates")
            # assure that each item is only assigned to one knapsack
            index = 0
            for j in range(len(seed_data)):
                for i in range(knapsacks):
                    if (individual_list[i][j] == 1) and (i < knapsacks - 1):
                        index = i
                        for i in range(index + 1, knapsacks):
                            individual_list[i][j] = 0
                        break
            # for i in range (knapsacks):
            #   print(individual_list[i])

            return individual_list

        def crossover(parent_1, parent_2):  # one crossover for each knapsack
            """Crossover (mate) two parents to produce two children.

            :param parent_1: candidate solution representation (list)
            :param parent_2: candidate solution representation (list)
            :returns: tuple containing two children

            """
            child_1_list = []
            child_2_list = []
            for k in range(knapsacks):
                index_item = random.randrange(1, len(parent_1[k]))
                # print("index_item", index_item)
                child_1 = parent_1[k][:index_item] + parent_2[k][index_item:]
                child_2 = parent_2[k][:index_item] + parent_1[k][index_item:]
                child_1_list.append(child_1)
                child_2_list.append(child_2)
            '''print("parent_1", parent_1)
            print("parent_2", parent_2)
            print("crossover before")
            print("child_1_list")
            for i in range (knapsacks):
              print(child_1_list[i])
            print("child_2_list")
            for i in range (knapsacks):
              print(child_2_list[i])'''

            # assure that each item is only assigned to one knapsack
            index = 0
            for j in range(len(child_1_list[0])):
                for i in range(knapsacks):
                    if (child_1_list[i][j] == 1) and (i < knapsacks - 1):
                        index = i
                        for i in range(index + 1, knapsacks):
                            child_1_list[i][j] = 0
                        break
            for j in range(len(child_2_list[0])):
                for i in range(knapsacks):
                    if (child_2_list[i][j] == 1) and (i < knapsacks - 1):
                        index = i
                        for i in range(index + 1, knapsacks):
                            child_2_list[i][j] = 0
                        break
            '''print("crossover after")
            print("child_1_list")
            for i in range (knapsacks):
              print(child_1_list[i])
            print("child_2_list")
            for i in range (knapsacks):
              print(child_2_list[i])'''

            return child_1_list, child_2_list

        def mutate(individual):  # one mutation for each knapsack
            """Reverse the bit of a random index in an individual."""

            '''print("mutation")
            for i in range (knapsacks):
              print(individual[i])'''

            for k in range(knapsacks):
                mutate_index = random.randrange(len(individual[k]))
                # print("mutate_index", mutate_index)
                if individual[k][mutate_index] == 1:
                    # mutation from 1 to 0
                    individual[k][mutate_index] = 0
                else:
                    # assure that each item is only assigned to one knapsack
                    for i in range(knapsacks):
                        individual[i][mutate_index] = 0
                    # mutation from 0 to 1
                    individual[k][mutate_index] = 1

            '''print("mutation after")
            for i in range (knapsacks):
              print(individual[i])'''

        def random_selection(population):
            """Select and return a random member of the population."""
            return random.choice(population)

        def tournament_selection(population):
            """Select a random number of individuals from the population and
            return the fittest member of them all.
            """
            if self.tournament_size == 0:
                self.tournament_size = 2
            members = random.sample(population, self.tournament_size)
            members.sort(
                key=attrgetter('fitness'), reverse=self.maximise_fitness)
            return members[0]

        self.fitness_function = None
        self.tournament_selection = tournament_selection
        self.tournament_size = self.population_size // 10
        self.random_selection = random_selection
        self.create_individual = create_individual
        self.crossover_function = crossover
        self.mutate_function = mutate
        self.selection_function = self.tournament_selection

    def create_initial_population(self):
        """Create members of the first population randomly.
        """
        initial_population = []
        for _ in range(self.population_size):
            genes = self.create_individual(self.seed_data)
            individual = Chromosome(genes)
            initial_population.append(individual)
        self.current_generation = initial_population

    def calculate_population_fitness(self):
        """Calculate the fitness of every member of the given population using
        the supplied fitness_function.
        """
        for individual in self.current_generation:
            individual.fitness = self.fitness_function(
                individual.genes, self.seed_data)

    def rank_population(self):
        """Sort the population by fitness according to the order defined by
        maximise_fitness.
        """
        self.current_generation.sort(
            key=attrgetter('fitness'), reverse=self.maximise_fitness)

    def create_new_population(self):
        """Create a new population using the genetic operators (selection,
        crossover, and mutation) supplied.
        """
        new_population = []
        elite = copy.deepcopy(self.current_generation[0])
        selection = self.selection_function

        while len(new_population) < self.population_size:
            parent_1 = copy.deepcopy(selection(self.current_generation))
            parent_2 = copy.deepcopy(selection(self.current_generation))

            child_1, child_2 = parent_1, parent_2
            child_1.fitness, child_2.fitness = 0, 0

            can_crossover = random.random() < self.crossover_probability
            can_mutate = random.random() < self.mutation_probability

            if can_crossover:
                child_1.genes, child_2.genes = self.crossover_function(
                    parent_1.genes, parent_2.genes)

            if can_mutate:
                self.mutate_function(child_1.genes)
                self.mutate_function(child_2.genes)

            new_population.append(child_1)
            if len(new_population) < self.population_size:
                new_population.append(child_2)

        if self.elitism:
            new_population[0] = elite

        self.current_generation = new_population
        #self.current_generation = new_population[1]

    def create_first_generation(self):
        """Create the first population, calculate the population's fitness and
        rank the population by fitness according to the order specified.
        """
        self.create_initial_population()
        self.calculate_population_fitness()
        self.rank_population()

    def create_next_generation(self):
        """Create subsequent populations, calculate the population fitness and
        rank the population by fitness in the order specified.
        """
        self.create_new_population()
        self.calculate_population_fitness()
        self.rank_population()

    def run(self):
        """Run (solve) the Genetic Algorithm."""
        self.create_first_generation()

        for _ in range(1, self.generations):
            self.create_next_generation()

    def best_individual(self):
        """Return the individual with the best fitness in the current
        generation.
        """
        best = self.current_generation[0]
        return (best.fitness, best.genes)

    def last_generation(self):
        """Return members of the last generation as a generator function."""
        return ((member.fitness, member.genes) for member
                in self.current_generation)


class Chromosome(object):
    """ Chromosome class that encapsulates an individual's fitness and solution
    representation.
    """

    def __init__(self, genes):
        """Initialise the Chromosome."""
        self.genes = genes
        self.fitness = 0

    def __repr__(self):
        """Return initialised Chromosome representation in human readable form.
        """
        return repr((self.fitness, self.genes))

# Genetic Algorithm, multiple n-dimensional 0-1 Knapsack with PYEASYGA

In [78]:
# define a fitness function
def fitness(individual, data):  # individual = array of k arrays of 0s & 1s
    used_space = []  # used space for each dimension for each knapsack
    error = 0
    maxprofit = 0

    for k in range(NBR_KNAPSACKS):
        knapsack_used_space = []
        for i in range(DIMENSION + 1):
            knapsack_used_space.append(0)
        used_space.append(knapsack_used_space)

    for k in range(NBR_KNAPSACKS):
        for (selected, item) in zip(individual[k], data):  # selected = 0 or 1
            if selected:
                for i in range(1, DIMENSION + 1):
                    if (used_space[k][i] + item[
                        i]) > MAX_SPACE:  # if one dimension is higher that the capacity -> error = 1
                        error = 1
                    if error < 1:
                        used_space[k][i] += item[i]
                if error < 1:
                    used_space[k][0] += item[0]  # profit
                else:
                    error = 0

    # calculate maxprofit as sum of the profits of all knapsacks
    for k in range(NBR_KNAPSACKS):
        maxprofit = maxprofit + used_space[k][0]

    avg_used_space = 0
    for k in range(NBR_KNAPSACKS):
        for i in range(1, DIMENSION + 1):
            avg_used_space += used_space[k][i]
        avg_used_space = avg_used_space / DIMENSION
    avg_used_space = round(avg_used_space / NBR_KNAPSACKS, 5)

    return maxprofit, avg_used_space, used_space[1:]  # maxprofit, avg_used_space, used_space

In [79]:
# crossover probability = 0.8, pop size = 150, generations = 75, mutation probability = 0.2
def genetigAlg_multiple(data):
    ga = MultipleGeneticAlgorithm(  # initialise the GA with data & more
        data,
        knapsacks=NBR_KNAPSACKS,
        population_size=MU,  # number of individuals to select for the next generation
        generations=NGEN,  # number of generations
        crossover_probability=0.8,  # probability that an offspring is produced by crossover
        mutation_probability=MUTPB,  # probability that an offspring is produced by mutation
        elitism=True,
        maximise_fitness=True)
    ga.fitness_function = fitness  # set the GA's fitness function
    ga.run()  # run the GA
    # print("best individual", ga.best_individual())  # print the GA's best solution
    # print("2: maxprofit", ga.best_individual()[0][0], ", profit", ga.best_individual()[0][3])
    return ga.best_individual()  # returns (best.fitness (maxprofit, avg_used_space, used_space), best.genes)

In [80]:
avgtimePYEASYGA = 0
resultsPYEASYGA = [0, 0]  # maxprofit, avg_used_space

for j in range(NBR_EXPERIMENTS):
    print(j + 1)
    my_data_nd = create_items_nd().values()

    # get time
    elapsed = calcTime(genetigAlg_multiple, my_data_nd)
    avgtimePYEASYGA = avgtimePYEASYGA + elapsed

    result = genetigAlg_multiple(my_data_nd)
    # get maxprofit
    resultsPYEASYGA[0] = resultsPYEASYGA[0] + result[0][0]
    # get average used space
    resultsPYEASYGA[1] = resultsPYEASYGA[1] + result[0][1]

avgtimePYEASYGA = round(avgtimePYEASYGA / NBR_EXPERIMENTS, 5)
avgprofit = round(resultsPYEASYGA[0] / NBR_EXPERIMENTS, 5)
avgspace = round(resultsPYEASYGA[1] / NBR_EXPERIMENTS, 5)

print("Statistical evaluation of multiple n-dimensional genetic 0-1 knapsack (PYEASYGA)")
print("\t\t\t\t\tmillisec\tmaxprofit\taverage used space")
print("multiple PYEASYGA\t", avgtimePYEASYGA, "\t", avgprofit, "\t\t", avgspace, "\t")

1
Statistical evaluation of multiple n-dimensional genetic 0-1 knapsack (PYEASYGA)
					millisec	maxprofit	average used space
multiple PYEASYGA	 1526.65655 	 1693.0 		 9.19746 	


# Improved pyeasyga module for multiple knapsack
The existing code of the pyeasyga module is modified to improve the genetic algorithm for the multiple multidimensional knapsack problem.

In [81]:
# -*- coding: utf-8 -*-
"""
    pyeasyga module for multiple knapsack: class MultipleImprovedGeneticAlgorithm

"""

import random
import copy
from operator import attrgetter

from six.moves import range


class MultipleImprovedGeneticAlgorithm(object):
    """Genetic Algorithm class.

    This is the main class that controls the functionality of the Genetic
    Algorithm.

    A simple example of usage:

    >>> # Select only two items from the list and maximise profit
    >>> from pyeasyga.pyeasyga import GeneticAlgorithm
    >>> input_data = [('pear', 50), ('apple', 35), ('banana', 40)]
    >>> easyga = GeneticAlgorithm(input_data)
    >>> def fitness (member, data):
    >>>     return sum([profit for (selected, (fruit, profit)) in
    >>>                 zip(member, data) if selected and
    >>>                 member.count(1) == 2])
    >>> easyga.fitness_function = fitness
    >>> easyga.run()
    >>> print easyga.best_individual()

    """

    def __init__(self,
                 seed_data,
                 max_capacity,    # for the improved create_individual function
                 number_of_dimensions=1,    # for the improved create_individual function, n-dimensional
                 knapsacks=1,       # for multiple multidimensional knapsack
                 population_size=50,
                 generations=100,
                 crossover_probability=0.8,
                 mutation_probability=0.2,
                 elitism=True,
                 maximise_fitness=True):
        """Instantiate the Genetic Algorithm.

        :param seed_data: input data to the Genetic Algorithm
        :type seed_data: list of objects
        :param int population_size: size of population
        :param int generations: number of generations to evolve
        :param float crossover_probability: probability of crossover operation
        :param float mutation_probability: probability of mutation operation

        """

        self.seed_data = seed_data
        self.max_capacity = max_capacity    # for the improved create_individual function
        self.number_of_dimensions = number_of_dimensions    # for the improved create_individual function, n-dimensional
        self.knapsacks = knapsacks    # for multiple multidimensional knapsack
        self.population_size = population_size
        self.generations = generations
        self.crossover_probability = crossover_probability
        self.mutation_probability = mutation_probability
        self.elitism = elitism
        self.maximise_fitness = maximise_fitness

        self.current_generation = []

        def create_individual(seed_data):
            """Create a candidate solution representation.

            e.g. for a bit array representation:

            >>> return [random.randint(0, 1) for _ in range(len(data))]

            :param seed_data: input data to the Genetic Algorithm
            :type seed_data: list of objects
            :returns: candidate solution representation as a list

            """
            feasible = True
            individual_list = []
            random.shuffle(seed_data)
              
            for k in range (knapsacks):
              knapsack_list = []
              capacity = []
              for i in range(number_of_dimensions):
                capacity.append(0)
              for item in seed_data:
                r = random.randint(0, 1)
                if (r):
                  for i in range(number_of_dimensions):
                    capacity[i] = capacity[i] + item[i+1]
                    if capacity[i] > self.max_capacity:
                      knapsack_list.append(0)
                      feasible = False
                      break
                if not feasible:
                  break
                knapsack_list.append(r)
              for i in range(len(seed_data)-len(knapsack_list)):
                knapsack_list.append(0)
              individual_list.append(knapsack_list)
            
            # assure that each item is only assigned to one knapsack
            index = 0
            for j in range (len(seed_data)):
              for k in range (knapsacks):
                if (individual_list[k][j] == 1) and (k < knapsacks-1):
                  index = k
                  for i in range (index+1, knapsacks):
                    individual_list[i][j] = 0
                  break
            # print("individual_list", individual_list)
            return individual_list

        def crossover(parent_1, parent_2):                            # one crossover for each knapsack
            """Crossover (mate) two parents to produce two children.

            :param parent_1: candidate solution representation (list)
            :param parent_2: candidate solution representation (list)
            :returns: tuple containing two children

            """
            child_1_list = []
            child_2_list = []
            for k in range (knapsacks):
              index_item = random.randrange(1, len(parent_1[k]))
              # print("index_item", index_item)
              child_1 = parent_1[k][:index_item] + parent_2[k][index_item:]
              child_2 = parent_2[k][:index_item] + parent_1[k][index_item:]
              child_1_list.append(child_1)
              child_2_list.append(child_2)
            '''print("parent_1", parent_1)
            print("parent_2", parent_2)
            print("crossover before")
            print("child_1_list")
            for i in range (knapsacks):
              print(child_1_list[i])
            print("child_2_list")
            for i in range (knapsacks):
              print(child_2_list[i])'''

            # assure that each item is only assigned to one knapsack
            index = 0
            for j in range (len(child_1_list[0])):
              for i in range (knapsacks):
                if (child_1_list[i][j] == 1) and (i < knapsacks-1):
                  index = i
                  for i in range (index+1, knapsacks):
                    child_1_list[i][j] = 0
                  break
            for j in range (len(child_2_list[0])):
              for i in range (knapsacks):
                if (child_2_list[i][j] == 1) and (i < knapsacks-1):
                  index = i
                  for i in range (index+1, knapsacks):
                    child_2_list[i][j] = 0
                  break
            '''print("crossover after")
            print("child_1_list")
            for i in range (knapsacks):
              print(child_1_list[i])
            print("child_2_list")
            for i in range (knapsacks):
              print(child_2_list[i])'''

            return child_1_list, child_2_list

        def mutate(individual):                             # one mutation for each knapsack
            """Reverse the bit of a random index in an individual."""

            '''print("mutation")
            for i in range (knapsacks):
              print(individual[i])'''

            for k in range (knapsacks):
              mutate_index = random.randrange(len(individual[k]))
              # print("mutate_index", mutate_index)
              if (individual[k][mutate_index] == 1):
                # mutation from 1 to 0
                individual[k][mutate_index] = 0  
              else: 
                # assure that each item is only assigned to one knapsack
                for i in range (knapsacks):
                  individual[i][mutate_index] = 0
                # mutation from 0 to 1
                individual[k][mutate_index] = 1

            '''print("mutation after")
            for i in range (knapsacks):
              print(individual[i])'''

        def random_selection(population):
            """Select and return a random member of the population."""
            return random.choice(population)

        def tournament_selection(population):
            """Select a random number of individuals from the population and
            return the fittest member of them all.
            """
            if self.tournament_size == 0:
                self.tournament_size = 2
            members = random.sample(population, self.tournament_size)
            members.sort(
                key=attrgetter('fitness'), reverse=self.maximise_fitness)
            return members[0]

        self.fitness_function = None
        self.tournament_selection = tournament_selection
        self.tournament_size = self.population_size // 10
        self.random_selection = random_selection
        self.create_individual = create_individual
        self.crossover_function = crossover
        self.mutate_function = mutate
        self.selection_function = self.tournament_selection

    def create_initial_population(self):
        """Create members of the first population randomly.
        """
        initial_population = []
        for _ in range(self.population_size):
            genes = self.create_individual(self.seed_data)
            individual = Chromosome(genes)
            initial_population.append(individual)
        self.current_generation = initial_population

    def calculate_population_fitness(self):
        """Calculate the fitness of every member of the given population using
        the supplied fitness_function.
        """
        for individual in self.current_generation:
            individual.fitness = self.fitness_function(
                individual.genes, self.seed_data)

    def rank_population(self):
        """Sort the population by fitness according to the order defined by
        maximise_fitness.
        """
        self.current_generation.sort(
            key=attrgetter('fitness'), reverse=self.maximise_fitness)

    def create_new_population(self):
        """Create a new population using the genetic operators (selection,
        crossover, and mutation) supplied.
        """
        new_population = []
        elite = copy.deepcopy(self.current_generation[0])
        selection = self.selection_function

        while len(new_population) < self.population_size:
            parent_1 = copy.deepcopy(selection(self.current_generation))
            parent_2 = copy.deepcopy(selection(self.current_generation))

            child_1, child_2 = parent_1, parent_2
            child_1.fitness, child_2.fitness = 0, 0

            can_crossover = random.random() < self.crossover_probability
            can_mutate = random.random() < self.mutation_probability

            if can_crossover:
                child_1.genes, child_2.genes = self.crossover_function(
                    parent_1.genes, parent_2.genes)

            if can_mutate:
                self.mutate_function(child_1.genes)
                self.mutate_function(child_2.genes)

            new_population.append(child_1)
            if len(new_population) < self.population_size:
                new_population.append(child_2)

        if self.elitism:
            new_population[0] = elite

        self.current_generation = new_population
        #self.current_generation = new_population[1]

    def create_first_generation(self):
        """Create the first population, calculate the population's fitness and
        rank the population by fitness according to the order specified.
        """
        self.create_initial_population()
        self.calculate_population_fitness()
        self.rank_population()

    def create_next_generation(self):
        """Create subsequent populations, calculate the population fitness and
        rank the population by fitness in the order specified.
        """
        self.create_new_population()
        self.calculate_population_fitness()
        self.rank_population()

    def run(self):
        """Run (solve) the Genetic Algorithm."""
        self.create_first_generation()

        for _ in range(1, self.generations):
            self.create_next_generation()

    def best_individual(self):
        """Return the individual with the best fitness in the current
        generation.
        """
        best = self.current_generation[0]
        return (best.fitness, best.genes)

    def last_generation(self):
        """Return members of the last generation as a generator function."""
        return ((member.fitness, member.genes) for member
                in self.current_generation)


class Chromosome(object):
    """ Chromosome class that encapsulates an individual's fitness and solution
    representation.
    """
    def __init__(self, genes):
        """Initialise the Chromosome."""
        self.genes = genes
        self.fitness = 0

    def __repr__(self):
        """Return initialised Chromosome representation in human readable form.
        """
        return repr((self.fitness, self.genes))

# Improved GA, multiple n-dimensional 0-1 Knapsack

In [82]:
# define a fitness function
def fitness(individual, data):    # individual = array of k arrays of 0s & 1s
    used_space = []   # used space for each dimension for each knapsack
    error = 0
    maxprofit = 0

    for k in range(NBR_KNAPSACKS):
      knapsack_used_space = []
      for i in range(DIMENSION+1):
        knapsack_used_space.append(0)
      used_space.append(knapsack_used_space)

    for k in range(NBR_KNAPSACKS):
      for (selected, item) in zip(individual[k], data):  # selected = 0 or 1
          if selected:
              for i in range(1, DIMENSION+1):
                if (used_space[k][i]+item[i]) > MAX_SPACE:  # if one dimension is higher that the capacity -> error = 1
                  error = 1                  
                if error < 1:
                  used_space[k][i] += item[i]
              if error < 1:
                used_space[k][0] += item[0]                # profit
              else:
                error = 0

    # calculate maxprofit as sum of the profits of all knapsacks
    for k in range (NBR_KNAPSACKS):
      maxprofit = maxprofit + used_space[k][0]

    avg_used_space = 0
    for k in range (NBR_KNAPSACKS):
      for i in range (1, DIMENSION+1):
        avg_used_space += used_space[k][i]
      avg_used_space = avg_used_space / DIMENSION
    avg_used_space = round(avg_used_space / NBR_KNAPSACKS, 5)

    return maxprofit, avg_used_space, used_space[1:]  # maxprofit, avg_used_space, used_space

In [83]:
# crossover probability = 0.8, pop size = 150, generations = 75, mutation probability = 0.2
def improvedGenetigAlg_multiple(data):
  ga = MultipleImprovedGeneticAlgorithm(         # initialise the GA with data & more
          data,
          max_capacity=MAX_SPACE,
          number_of_dimensions=DIMENSION,
          knapsacks=NBR_KNAPSACKS,
          population_size=MU,             # number of individuals to select for the next generation
          generations=NGEN,               # number of generations
          crossover_probability=0.8,     # probability that an offspring is produced by crossover
          mutation_probability=MUTPB,     # probability that an offspring is produced by mutation
          elitism=True,
          maximise_fitness=True)
  ga.fitness_function = fitness               # set the GA's fitness function
  ga.run()                                    # run the GA
  # print("best individual", ga.best_individual())  # print the GA's best solution
  # print("2: maxprofit", ga.best_individual()[0][0], ", profit", ga.best_individual()[0][3])
  return ga.best_individual()                # returns (best.fitness (maxprofit, avg_used_space, used_space), best.genes)

In [84]:
avgtimePYEASYGA_improved = 0
resultsPYEASYGA_improved = [0,0] # maxprofit, avg_used_space

for j in range (NBR_EXPERIMENTS):
  print(j+1)
  my_data_nd = create_items_nd().values()
  # print("my data", my_data_nd)

  # get time
  elapsed = calcTime(improvedGenetigAlg_multiple, list(my_data_nd))
  avgtimePYEASYGA_improved = avgtimePYEASYGA_improved + elapsed

  result = improvedGenetigAlg_multiple(list(my_data_nd))
  # get maxprofit
  resultsPYEASYGA_improved[0] = resultsPYEASYGA_improved[0] + result[0][0]
  # get average used space
  resultsPYEASYGA_improved[1] = resultsPYEASYGA_improved[1] + result[0][1]

avgtimePYEASYGA_improved = round(avgtimePYEASYGA_improved / NBR_EXPERIMENTS, 5)
avgprofit_improved = round(resultsPYEASYGA_improved[0] / NBR_EXPERIMENTS, 5)
avgspace_improved = round(resultsPYEASYGA_improved[1] / NBR_EXPERIMENTS, 5)

print("Statistical evaluation of multiple n-dimensional improved genetic 0-1 knapsack (PYEASYGA)")
print("\t\t\t\t\tmillisec\tmaxprofit\taverage used space")
print("multiple PYEASYGA\t", avgtimePYEASYGA_improved, "\t", avgprofit_improved, "\t\t", avgspace_improved, "\t")

1
Statistical evaluation of multiple n-dimensional improved genetic 0-1 knapsack (PYEASYGA)
					millisec	maxprofit	average used space
multiple PYEASYGA	 1137.70286 	 2064.0 		 8.10768 	
